In [3]:
import numpy as np
import tensorflow as tf
from PIL import Image, ImageTk
import tkinter as tk
import cv2

In [11]:
# Load the main model (adjusting for the 28x28 image input size)
loaded_model = tf.keras.models.load_model("../notebook/best_model.keras")


In [12]:
# Counter for each letter A-Z
ct = {chr(i): 0 for i in range(65, 91)}
blank_flag = 0
current_symbol = "Empty"
word = ""
str_text = ""

In [13]:
# Setup tkinter GUI
root = tk.Tk()
root.title("Image Prediction")
panel = tk.Label(root)
panel.pack(padx=10, pady=10)
panel2 = tk.Label(root)
panel2.pack(padx=10, pady=10)
T = tk.Label(root)
T.pack()
panel3 = tk.Label(root)
panel3.pack(padx=10, pady=10)

In [20]:
# OpenCV video capture
cap = cv2.VideoCapture(0)

In [21]:
# Load Haar Cascade for face detection (substitute with face detection for testing)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(gray, (28, 28))
    img_array = np.array(img, dtype=np.float32)
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    img_array = np.reshape(img_array, (1, 28, 28, 1))  # Reshape for the model input
    return img_array

def predict(img):
    global current_symbol, word, blank_flag, str_text, ct

    try:
        result = loaded_model.predict(img)
        if result.shape[1] != 26:  # Ensure the result has 26 elements for A-Z
            raise ValueError("Unexpected model output shape")

        prediction = {
            chr(65 + i): result[0][i] for i in range(26)
        }

        # Sort predictions in descending order
        prediction = sorted(prediction.items(), key=lambda x: x[1], reverse=True)
        current_symbol = prediction[0][0]
        confidence = prediction[0][1] * 100  # Prediction percentage

        # Update letter count and word
        if current_symbol == " ":
            for i in ct:
                ct[i] = 0
        ct[current_symbol] += 1

        if ct[current_symbol] > 60:
            if current_symbol != " ":
                word += current_symbol
            ct = {chr(i): 0 for i in range(65, 91)}

        if current_symbol == " ":
            if blank_flag == 0:
                blank_flag = 1
                if len(str_text) > 0 and str_text[-1] != " ":
                    str_text += " "
                str_text += word
                word = ""
        else:
            blank_flag = 0

        # Update GUI with the predictions
        panel2.config(text=current_symbol, font=("Courier", 50))
        panel3.config(text=word, font=("Courier", 40))
        T.config(text=str_text, font=("Courier", 40))

        return confidence

    except Exception as e:
        print(f"An error occurred during prediction: {e}")
        return 0

def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.flip(frame, 1)  # Flip frame horizontally for a more natural selfie view

        # Face detection (substitute with face detection as a fallback)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            confidence = 0
            roi = frame[y:y+h, x:x+w]
            img_array = preprocess_frame(roi)
            confidence = predict(img_array)  # Get prediction confidence

            # Draw rectangle around detected face and put the confidence percentage
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, f"{current_symbol}: {confidence:.2f}%",
                        (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        img = Image.fromarray(img)
        img_tk = ImageTk.PhotoImage(image=img)

        panel.config(image=img_tk)
        panel.image = img_tk  # Keep a reference to avoid garbage collection

    root.after(10, update_frame)  # Schedule next frame update

update_frame()
root.mainloop()

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

RuntimeError: Too early to create image: no default root window